# Dropout Prediction EDA

In [1]:
import pandas as pd
import numpy as np
from hashlib import sha1
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

In [2]:
alt.renderers.enable('mimetype')

data_path = '../data/raw/data.csv'
sep = ';'

NameError: name 'RendererRegistry' is not defined

In [ ]:
df = pd.read_csv(data_path, sep=sep)
df = df.rename(columns={'Nacionality': 'Nationality'})
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df = df.dropna()
df.shape

The data-set consists of 4424 records with 35 attributes and contains no missing values. The distribution and statistics are above and below.

In [ ]:
df.info()

In [ ]:

sns.set(font_scale=1.4)
ax = df['Target'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
ax.bar_label(ax.containers[0])
plt.xlabel("Target", labelpad=14)
plt.ylabel("Count of Student", labelpad=14)
plt.title("Count of Student by Target", y=1.02);

From above plot, we can see this problem was three-category classification task, and there exists strong imbalance between those three classes. The class, Graduate, has the majority count which is around 50% of the records and Dropout has 32% of total records. The Enrolled only has 18% of total records. Thus, during our training, we need to find a way to fix this imbalance issues.

## Dropping Enrolled Student

In [ ]:
df = df.drop(df[df.Target == 'Enrolled'].index)
df.shape

In [ ]:
df.columns

In [ ]:

numeric_cols = ['Marital status', 'Nationality','Displaced','Gender',
                'Age at enrollment', 'International',
                "Mother's qualification", "Father's qualification",
       "Mother's occupation", "Father's occupation",
                
       'Educational special needs', 'Debtor',
       'Tuition fees up to date',  'Scholarship holder',
        'Unemployment rate','Inflation rate', 'GDP',
                
        'Application mode', 'Application order', 'Course',
                
       'Daytime/evening attendance\t', 'Previous qualification',
       #'Previous qualification (grade)', 
       # 'Admission grade',
        
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd sem (grade)',
       'Curricular units 2nd sem (without evaluations)', ]
df[numeric_cols].corr('spearman').style.background_gradient()

In [ ]:
macroeconomic_col = ['Unemployment rate','Inflation rate', 'GDP']

for i in range(0, len(macroeconomic_col)):

    sns.catplot(x='Target', y=macroeconomic_col[i], data=df, kind="box", aspect=1.5)
    plt.title("Box Plot of " + macroeconomic_col[i])
    


In [ ]:
sns.scatterplot(x='Unemployment rate', y='Inflation rate', hue="Target", data=df)
plt.title("Relationship between Unemployment Rate, Inflation Rate and target")
plt.show()

In [ ]:
numeric_cols = ['Age at enrollment',
                'Unemployment rate','Inflation rate', 'GDP',
        'Application mode', 'Application order' 
        ]

fig, axs = plt.subplots(2,3, figsize=(15, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.5)

axs = axs.ravel()

for i in range(0, len(numeric_cols)):

    axs[i].hist(df[numeric_cols[i]], bins=20)
    axs[i].set_title("Histogram of " + numeric_cols[i])

In [ ]:
sns.pairplot(df[['Age at enrollment',
                'Unemployment rate','Inflation rate', 'GDP',
        'Application mode', 'Application order' , 'Target']],
            hue='Target')

In [ ]:
numeric_cols = df.select_dtypes('number').columns.tolist()
numeric_cols

# performance of two semester vs Target


df_new = df[['Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)', 'Target']]

first_sem = ['Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)']

second_sem = ['Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)']

plot = alt.Chart(df_new).mark_point().encode(
     alt.X(alt.repeat('column'), type='quantitative'),
     alt.Y(alt.repeat('row'), type='quantitative'),
     color = 'Target'
).properties(
    width=200,
    height=200
).repeat(
    row = first_sem,
    column = second_sem
)

plot